In [28]:
# print version / should be 1.6.3
println(versioninfo())

# load centralized paths dictionary
using YAML
PATHS_FILE = "../../../PATHS.yml"
PATHS = YAML.load_file(PATHS_FILE);

"""
    load_path(keys::Vector{String}):String

Return the absolute path for a requested item within the nested PATHS dictionary.
"""
function loadpath(keys::Vector{String}):String
    # recusively assemble paths from keys
    requested_path = foldl((x, y) -> getindex(x, y), keys; init=PATHS)
    # rephrase abs path from relative paths to be platform independent.
    realpath(joinpath(splitdir(realpath(PATHS_FILE))[1], requested_path))
end;

Julia Version 1.6.3
Commit ae8452a9e0 (2021-09-23 17:34 UTC)
Platform Info:
  OS: Linux (x86_64-pc-linux-gnu)
  CPU: Intel(R) Xeon(R) CPU E5-2630 v4 @ 2.20GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-11.0.1 (ORCJIT, broadwell)
Environment:
  __LMOD_REF_COUNT_JULIA_DEPOT_PATH = /home/biotoml/.julia:1;/sw/comp/julia/1.6.3/rackham/lib/glob_pkg:1
  JULIA_DEPOT_PATH = /home/biotoml/lsm/src/.julia/v1.6_depot:/home/biotoml/.julia:/sw/comp/julia/1.6.3/rackham/lib/glob_pkg
  __LMOD_REF_COUNT_JULIA_LOAD_PATH = @:1;@v#.#:1;@stdlib:1;/sw/comp/julia/1.6.3/rackham/lib/glob_pkg/environments/v1.6:1
  JULIA_PROJECT = /crex/proj/snic2022-23-321/private/thomas/ext_src/01_julia_land_v01/Land/Project.toml
  JULIA_ROOT = /sw/comp/julia/1.6.3/rackham
  JULIA_LOAD_PATH = @:@v#.#:@stdlib:/sw/comp/julia/1.6.3/rackham/lib/glob_pkg/environments/v1.6
nothing


In [29]:
# we load the project path as it is used by CliMA Land v01 papers
using Pkg
PROJECT_PATH = loadpath(["EXTERNAL", "01_CLIMA_LAND_V01", "ROOT"])
Pkg.activate(PROJECT_PATH)

  Activating environment at `/crex/proj/snic2022-23-321/private/thomas/ext_src/01_julia_land_v01/Land/Project.toml`


In [116]:
using LazyArtifacts
using Dates

using CanopyLayers: BLUE, EVI, FourBandsFittingHybrid, NDVI, NIR, NIRv, NIRvES, RED, SIF_740, SIF_WL, SIF_fluxes!, canopy_fluxes!, canopy_geometry!, canopy_matrices!, fit_soil_mat!, short_wave!, soil_albedos
using DataFrames: DataFrame
using GriddingMachine: lat_ind, lon_ind
using Photosynthesis: AbstractPhotoModelParaSet, AirLayer, leaf_ETR!, GCO₂Mode, leaf_photosynthesis!
using PkgUtility: numerical∫, read_csv, save_csv!
using PlantHydraulics: soil_p_25_swc, temperature_effects!, AbstractPlantOrganism, VanGenuchten, create_grass
using SoilPlantAirContinuum: SPACMono, initialize_spac_canopy!, update_Cab!, update_LAI!, zenith_angle
using Statistics: mean
using StomataModels: BetaGLinearPsoil, CanopyLayer, ESMMedlyn, OSMWang, AbstractStomatalModel, OSMSperry, EmpiricalStomatalModel, GswDrive, gas_exchange!, gsw_control!, stomatal_conductance, update_leaf_TP!, β_factor
using UnPack: @unpack
using WaterPhysics: saturation_vapor_pressure

In [38]:
methods(create_grass)

# 1 method for generic function "create_grass":
[1] create_grass(z_root::FT, z_canopy::FT, soil_bounds::Vector{FT}, air_bounds::Vector{FT}) where FT<:AbstractFloat in PlantHydraulics at /home/biotoml/.julia/packages/PlantHydraulics/Aztny/src/initialize/plant.jl:20

In [100]:
EXAMPLE = Dict(
    # general
    "latitude" => - 105.4,
    "longitude" => 54.6,
    "date" => "2015-02-05T05:00:00",
    "surface_area" => 1.789e+03,
    "elevation" => 396.4,
    
    # winds
    "u10" => 2.523,
    "v10" => -0.9888,
    
    # temperatures
    "d2m" => 255.0,
    "t2m" => 257.6,
    "stl1" => 271.1,
    "stl2" => 271.7,
    "stl3" => 273.3,
    "stl4" => 275.4,
    "skt" => 256.9,
    
    # moisture
    "evavt" => 1.37e-06,
    
    # pressure
    "sp" => 9.467e+04,
    
    # waves
    "fal" => 0.3225,
    "sshf" => 1.372e+06,
    "soil_color" => 19.64,
    "msdrswrf" => 131.052,
    "msdwswrf" => 272.068,
    
    # plant specifications
    "lai_hv" => 1.422,
    "lai_lv" => 0.0,
    "lai" => 0.8437,
    "biomass" => 0.03891,
    "canopy_height" => 9.5,
    "chlorophyll" => 14.0,
    "clumping_index" => 0.5567,
    "vcmax_rubisco" => 65.34,
    "tree_density" => 6.494e+04,
    
    # soil hydraulics
    "src" => 0.0002756,
    "soil_mswc1" => 0.7119,
    "soil_mswc2" => 0.6992,
    "soil_mswc3" => 0.4053,
    "soil_mswc4" => 0.4456,
    "soil_wcr1" => 0.03256,
    "soil_wcr2" => 0.0327,
    "soil_wcr3" => 0.04254,
    "soil_wcr4" => 0.02991,
    "soil_vga1" => 468.5,
    "soil_vga2" => 541.8,
    "soil_vga3" => 296.1,
    "soil_vga4" => 591.6,
    "soil_vgn1" => 1.142,
    "soil_vgn2" => 1.145,
    "soil_vgn3" => 1.179,
    "soil_vgn4" => 1.154,
    "soil_ksat1" => 202.0,
    "soil_ksat2" => 82.86,
    "soil_ksat3" => 63.88,
    "soil_ksat4" => 60.24,
    "swvl1" => 0.2048,
    "swvl2" => 0.1967,
    "swvl3" => 0.1708,
    "swvl4" => 0.224,
);



In [98]:
# function to create a SPAC node
function create_spac(
        lat::FT,  # core
        lon::FT,  # core
        elevation::FT,  # can be computed from lat+lon
        
        # following components can be taken from GriddingMachine / ERA5
        wind_v10::FT,
        wind_u10::FT,
#        wind_d::FT,
#        wind_zs::Vector{FT},
#        winds::Vector{FT},
        mswc::Vector{FT},
        swc::Vector{FT},
#        p_soil::Vector{FT},
#        h_soil::FT,
        lai::FT;  # leaf area index from         
        
        _soil_bnds::AbstractVector{FT} = FT[0, -0.1, -0.35, -1, -3],  # 4 layers of soil with given boundaries (in m) as for JULES - can use ERA5 data
        _z_c::FT = 5.0,  # canopy height should be used from langHighresolutionCanopyHeight2022 or GriddingMachine
        _z_r::FT = -2,  # we set a default 2 m max root depth. Closest reference is fanHydrologicRegulationPlant2017 and https://wci.earth2observe.eu/thredds/catalog/usc/root-depth/catalog.html
        n_airlayers::Int = 20,  # not more than 127!
        
        # photomodel C3 or C4
        #  can be C3VJPModel, C3CytochromeModel, C4VJPModel
        # plant_hs::AbstractPlantOrganism{FT} = create_grass(_z_r, _z_c),  # vegetation_type - GrassLikeOrganism, TreeLikeOrganism, PalmLikeOrganism (init via create_*)
        
        # Stomatal models - we can try different models - optimization approaches were great in trials
        stomata_model::AbstractStomatalModel{FT} = OSMWang{FT}()
        
        
        ) where {FT<:AbstractFloat}
    _Δz::FT    = _z_c / n_airlayers;
    _air_bnds  = collect(FT, 0:_Δz:_z_c+2*_Δz);
    _elev::FT  = 100;

    # we compute vector length of wind given u and v component
    wind_z0 = sqrt(wind_v10^2 + wind_u10^2)
    
    # create a SPACMono struct
    _node = SPACMono{FT}(soil_bounds=_soil_bnds, air_bounds=_air_bnds, z_canopy=_z_c, z_root=_z_r, latitude=lat, longitude=lon, elevation=_elev, stomata_model=stomata_model);

    # initialize the canopy RT model
    initialize_spac_canopy!(_node);

    return _node, wind_z0

end;




In [51]:
FT = Float64

Float64

In [117]:
update_gsw!(clayer::CanopyLayer{FT}, sm::EmpiricalStomatalModel{FT}, photo_set::AbstractPhotoModelParaSet{FT}, envir::AirLayer{FT}, Δt::FT; β::FT=FT(1), τ::FT=FT(600)) where {FT<:AbstractFloat} = (
    # unpack values
    @unpack g_bc, g_bw, g_lc, g_lw, g_m, g_sc, g_sw, n_leaf = clayer;

    # calculate steady state values
    # assume τ = 10 minutes
    for _iLF in 1:n_leaf
        # TODO add the bug fix to SPAC or StomataModels
        # bug fix: upadte APAR per leaf
        clayer.ps.APAR = clayer.APAR[_iLF];
        leaf_ETR!(photo_set, clayer.ps);

        _gsw_ss = max(sm.g0, stomatal_conductance(sm, clayer, envir, β, _iLF));
        g_sw[_iLF] += (_gsw_ss - g_sw[_iLF]) / τ * Δt;

        # bug fix: update g_lw and others as well
        g_lw[_iLF] = 1 / ( 1/g_sw[_iLF]  + 1/g_bw[_iLF] );
        g_sc[_iLF] = g_sw[_iLF] / FT(1.6);
        g_lc[_iLF] = 1 / ( 1/g_sc[_iLF] + 1/g_m[_iLF] + 1/g_bc[_iLF] );
    end;

    return nothing
);

function update_gsw!(clayer::CanopyLayer{FT},
            sm::OSMWang{FT},
            photo_set::AbstractPhotoModelParaSet{FT},
            envir::AirLayer{FT},
            Δt::FT;
            τ::FT = FT(1e-6)) where {FT<:AbstractFloat} 
    # unpack values
    @unpack An, ec, g_bc, g_bw, g_lw, g_m, g_max, g_min, g_sw, n_leaf, p_sat,
            ps = clayer;
    @unpack p_atm, p_H₂O = envir;

    # update g_sw
    for iLF in 1:n_leaf
        _gsw = g_sw[iLF] .+ FT(0.001);
        _glw = 1 / ( 1/_gsw + 1/g_bw[iLF] );
        _gsc = _gsw / FT(1.6);
        _glc = 1 / ( 1/_gsc + 1/g_m[iLF] + 1/g_bc[iLF] );
        leaf_photosynthesis!(photo_set, ps, envir, GCO₂Mode(), _glc);
        _∂A   = ps.An - An[iLF];
        _e0   = g_lw[iLF] * (p_sat - p_H₂O) / p_atm;
        _e1   = _glw * (p_sat - p_H₂O) / p_atm;
        _∂E   = _e1 - _e0;
        _∂A∂E = _∂A / _∂E;
        _∂Θ∂E = FT(max(0.1, An[iLF]) / max(ec - _e0, 1e-7));

        # ensure that dgsw does not change too rapidly
        _Δgsw = (_∂A∂E - _∂Θ∂E) * τ * Δt;
        if _Δgsw > 0
            _Δgsw = min(_Δgsw, (g_max-g_sw[iLF]) / 4);
        else
            _Δgsw = max(_Δgsw, (g_min-g_sw[iLF]) / 4);
        end;
        g_sw[iLF] += _Δgsw;
    end;
end

update_gsw! (generic function with 3 methods)

In [119]:
### dict_features = EXAMPLE

# first create the SPAC
node, wind_z0 = create_spac(dict_features["latitude"],  # lat
                   dict_features["longitude"],  # lat
                   dict_features["elevation"],  # elevation
                   
                   # following components can be taken from GriddingMachine / ERA5
                   dict_features["v10"],  # lat
                   dict_features["u10"],  # elevation
                   [dict_features["soil_mswc1"], dict_features["soil_mswc2"],  # mswc
                        dict_features["soil_mswc3"], dict_features["soil_mswc4"]],
                   [dict_features["swvl1"], dict_features["swvl2"],  # swc
                        dict_features["swvl3"], dict_features["swvl4"]],
                   dict_features["lai"];  # leaf area index         
                   _soil_bnds = FT[0, -0.1, -0.35, -1, -3],  # 4 layers of soil with given boundaries (in m) as for JULES - can use ERA5 data
                   _z_c = dict_features["canopy_height"],  # canopy height
                   _z_r = -2.,  # we set a default 2 m max root depth.
                   n_airlayers = 20,  # not more than 127!
        
                   # photomodel C3 or C4
                   #  can be C3VJPModel, C3CytochromeModel, C4VJPModel
                   # plant_hs = create_grass,  # vegetation_type - GrassLikeOrganism, TreeLikeOrganism, PalmLikeOrganism (init via create_*)
                   
                   # Stomatal models - we can try different models - optimization approaches were great in trials
                   stomata_model = OSMWang{FT}()
        )

# get the date to obtain day of the year
date_data = DateTime(dict_features["date"], dateformat"y-m-dTH:M:S")


# unpack variables
@unpack angles, can_opt, can_rad, canopy_rt, envirs, f_SL, ga, in_rad, latitude, leaves_rt, n_canopy, photo_set, plant_hs, plant_ps, rt_con, rt_dim, soil_opt, stomata_model, wl_set = node;
@unpack nAzi, nIncl = canopy_rt;
_in_rad_bak = deepcopy(in_rad);
_in_dir = numerical∫(_in_rad_bak.E_direct, wl_set.dWL) / 1000;
_in_dif = numerical∫(_in_rad_bak.E_diffuse, wl_set.dWL) / 1000;
_nSL = nAzi * nIncl;
_beta_g = BetaGLinearPsoil{FT}();
_method = FourBandsFittingHybrid();
co2 = 40
_f_H₂O  = 0
_f_CO₂  = 0
_f_GPP  = 0
_f_APAR = 0

# compute vapor pressure deficit from dewpoint
vpd = get_vpd(dict_features["t2m"], dict_features["d2m"])


# to insert the LAI into all different structures, we use a given function
update_LAI!(node, dict_features["lai"]) 

# to insert the chlorophyll and carotinoids
for _leaf in node.leaves_rt
        _leaf.Car = dict_features["chlorophyll"] / 7; 
end;
update_Cab!(node, dict_features["chlorophyll"]);

# we update the roots soil hydraulics to van genuchten
for i in 1:4
    plant_hs.roots[i].sh = VanGenuchten(
        stype="soil",
        α=dict_features["soil_vga$i"],
        n=dict_features["soil_vgn$i"],
        m=1 - 1/dict_features["soil_vgn$i"],
        Θs=dict_features["soil_mswc$i"],
        Θr=dict_features["soil_wcr$i"])
end

# we do gpp prediction if radiation is given
_e_dir = dict_features["msdrswrf"];
_e_dif = dict_features["msdwswrf"]-dict_features["msdrswrf"];
if _e_dir + _e_dif > 10.
    # update soil water matrices per layer
    _svc_1 = plant_hs.roots[1].sh;
    _svc_2 = plant_hs.roots[2].sh
    _svc_3 = plant_hs.roots[3].sh
    _svc_4 = plant_hs.roots[4].sh
    _swc_1 = max(_svc_1.Θr+FT(0.001), FT(dict_features["swvl1"]));
    _swc_2 = max(_svc_2.Θr+FT(0.001), FT(dict_features["swvl2"]));
    _swc_3 = max(_svc_3.Θr+FT(0.001), FT(dict_features["swvl3"]));
    _swc_4 = max(_svc_4.Θr+FT(0.001), FT(dict_features["swvl4"]));

    _p_1   = soil_p_25_swc(_svc_1, _swc_1);
    _p_2   = soil_p_25_swc(_svc_2, _swc_2);
    _p_3   = soil_p_25_swc(_svc_3, _swc_3);
    _p_4   = soil_p_25_swc(_svc_4, _swc_4);
    
    # forward hydraulic pressure from soil into xylem
    plant_hs.roots[1].p_ups = _p_1;
    plant_hs.roots[2].p_ups = _p_2;
    plant_hs.roots[3].p_ups = _p_3;
    plant_hs.roots[4].p_ups = _p_4;

    # adjust soil surface opticals
    node.soil_opt.color = round(dict_features["soil_color"])
    node.soil_opt.T = dict_features["skt"]    
    
    # update soil albedo
    # if hyper
    fit_soil_mat!(node.soil_opt, node.wl_set, _swc_1, _method);
    # else
    #     _ρ_PAR,_ρ_NIR = soil_albedos(node.soil_opt.color, _swc_1, true);
    #     node.soil_opt.ρ_SW .= _ρ_NIR;
    #     node.soil_opt.ρ_SW[node.wl_set.iPAR] .= _ρ_PAR;
    #     node.soil_opt.ρ_SW_SIF .= node.soil_opt.ρ_SW[node.wl_set.iWLF];
    #     node.soil_opt.ε_SW .= 1 .- node.soil_opt.ρ_SW;
    # end;

    # update PAR related information
    in_rad.E_direct  .= _in_rad_bak.E_direct  .* _e_dir ./ _in_dir;
    in_rad.E_diffuse .= _in_rad_bak.E_diffuse .* _e_dif ./ _in_dif;
    angles.sza = min(88, zenith_angle(latitude, FT(daysinyear(date_data))));
    canopy_geometry!(canopy_rt, angles, can_opt, rt_con);
    canopy_matrices!(leaves_rt, can_opt);
    short_wave!(canopy_rt, can_opt, can_rad, in_rad, soil_opt, rt_con);
    canopy_fluxes!(canopy_rt, can_opt, can_rad, in_rad, soil_opt, leaves_rt, wl_set, rt_con);

    # calculate leaf level flux per canopy layer
    for _i_can in 1:n_canopy
        _iPS = plant_ps[_i_can];
        _iRT = n_canopy + 1 - _i_can;

        # calculate the fraction of sunlit and shaded leaves
        _f_view = (can_opt.Ps[_iRT] + can_opt.Ps[_iRT+1]) / 2;
        for iLF in 1:_nSL
            _iPS.APAR[iLF] = can_rad.absPAR_sunCab[(_iRT-1)*_nSL+iLF] * FT(1e6);
            _iPS.LAIx[iLF] = _f_view * f_SL[iLF];
        end;
        _iPS.APAR[end] = can_rad.absPAR_shadeCab[_iRT] * FT(1e6);
        _iPS.LAIx[end] = 1 - _f_view;
    end;
    
    # calculate leaf level flux per canopy layer
    for _i_can in 1:n_canopy
        _iEN = envirs[_i_can];
        _iHS = plant_hs.leaves[_i_can];
        _iPS = plant_ps[_i_can];

        # update environmental conditions
        _iEN.t_air = dict_features["t2m"];
        _iEN.p_atm = dict_features["sp"];
        _iEN.p_a   = _iEN.p_atm * co2 * 1e-6;
        _iEN.p_O₂  = _iEN.p_atm * 0.209;
        _iEN.p_sat = saturation_vapor_pressure(_iEN.t_air);
        _iEN.vpd   = vpd;
        _iEN.p_H₂O = _iEN.p_sat - _iEN.vpd;
        _iEN.RH    = _iEN.p_H₂O / _iEN.p_sat;
        _iEN.wind  = wind_z0;

        # prescribe leaf temperature
        _iPS.T = max(dict_features["t2m"], dict_features["skt"]);
        update_leaf_TP!(photo_set, _iPS, _iHS, _iEN);
        temperature_effects!(_iHS, _iPS.T);

        # iterate for 30 times to find steady state solution
        if _e_dir + _e_dif > 10
            for iter in 1:30
                # beta factor
                _pl = _iHS.p_dos;
                _β1 = β_factor(_iHS, _svc_1, _beta_g, _pl, _p_1, _swc_1);
                _β2 = β_factor(_iHS, _svc_2, _beta_g, _pl, _p_2, _swc_2);
                _β3 = β_factor(_iHS, _svc_3, _beta_g, _pl, _p_3, _swc_3);
                _β4 = β_factor(_iHS, _svc_4, _beta_g, _pl, _p_4, _swc_4);
                _βm = mean([_β1, _β2, _β3, _β4]);

                # calculate the photosynthetic rates
                gas_exchange!(photo_set, _iPS, _iEN, GswDrive());
                # TODO add soil water content corrected beta function
                # update_gsw!(_iPS, stomata_model, photo_set, _iEN, FT(120); β=_βm);
                update_gsw!(_iPS, stomata_model, photo_set, _iEN, FT(120));
                gsw_control!(photo_set, _iPS, _iEN);
            end;
        else
            # set g_sw and APAR to 0, and then gsw_control!
            _iPS.APAR .= 0;
            _iPS.g_sw .= 0;
            gsw_control!(photo_set, _iPS, _iEN);
        end;

    # update the flow rates
        _f_H₂O  += numerical∫(_iPS.g_lw, _iPS.LAIx) * (_iPS.p_sat - _iEN.p_H₂O) / _iEN.p_atm * _iPS.LA;
        _f_CO₂  += numerical∫(_iPS.An, _iPS.LAIx) * _iPS.LA;
        _f_GPP  += numerical∫(_iPS.Ag, _iPS.LAIx) * _iPS.LA;
        _f_APAR += numerical∫(_iPS.APAR, _iPS.LAIx) * FT(1e-6) * canopy_rt.LAI / canopy_rt.nLayer;
    end;

    # save the total flux
    F_H2O = _f_H₂O / ga;
    F_CO2 = _f_CO₂ / ga;
    F_GPP = _f_GPP / ga;
    
end;

In [120]:
F_GPP

0.4822665843477054

In [1]:
    "d2m" => 255.0,
    "t2m" => 257.6,

LoadError: syntax: extra token "julia" after end of expression

In [94]:
function get_relhumid(temp_dewpoint)
    t = temp_dewpoint
    ew = exp(-6096.9385*t^(-1) + 21.2409642 - 2.711193*10^(-2)*t + 1.673952*10^(-5)*t^2 + 2.433502*log(t))
end

function get_vpd(temp_ambient, temp_dewpoint)
    p_sat = saturation_vapor_pressure(temp_ambient)
    rel_humid = get_vpd(temp_dewpoint)/get_vpd(temp_ambient)
    vpd = p_sat * (1 - rel_humid)
end

get_vpd (generic function with 2 methods)

55.62047673716392